In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# TODO: We are using log10, discuss if we need to change the type of log that is used.


####################################
# Input Variables
####################################

# Input Variable 1 : -> IDF Threshold
idf_thrshold_no_of_failing_tcs =10 #Validation Set

# Input Variable 2 : -> Coverage Information
input_coverage_file_path = 'CombinedCoverageData.csv';

# Input Variable 3 : -> Y-value (from training data) Format
#df_y_values = pd.read_csv('y-values.csv')
df_y_values = pd.read_csv('Y_values10.csv', sep=';') # Training Set

In [17]:
# IDF Threshold Calculations
idf_values = []

for x in range(1, 10):
    idf_values.append(round(x/10 * idf_thrshold_no_of_failing_tcs))
    
print("IDF-Threshold Value = " + str(idf_values))

IDF-Threshold Value = [1, 2, 3, 4, 5, 6, 7, 8, 9]


In [18]:
# Loading Coverage File

coverage_information_main = pd.read_csv(input_coverage_file_path)

# Display top 5 rows o coverage file.
#coverage_information_main.head()

In [19]:
# Processing of Coverage Data

# 1. Sort By Test Case Classes - from Coverage Data

names_of_test_case_classes = coverage_information_main['Test Case'].unique()

#print("Number of Test Classes = " + str(names_of_test_case_classes.size) + "\n")
#print("Names of Test Classes = " + str(names_of_test_case_classes) + "\n")


# 2. Sort By Code Classes - from Coverage Data

names_of_code_classes = coverage_information_main['CLASS'].unique()

#print("Number of Code Classes = " + str(names_of_code_classes.size) + '\n')
#print("Names of Code Classes = " + str(names_of_code_classes)+ '\n')

group_by_test_classes  = coverage_information_main.groupby('Test Case')
group_by_code_classes  = coverage_information_main.groupby('CLASS')


no_of_tcs_impacted_by_each_class = group_by_test_classes.count()[['CLASS']]
no_of_classes_impacted_by_each_test_case = group_by_code_classes.count()[['Test Case']]


no_of_classes_impacted_by_each_test_case = no_of_classes_impacted_by_each_test_case.reset_index()
no_of_tcs_impacted_by_each_class = no_of_tcs_impacted_by_each_class.reset_index()


In [20]:
# Calculate size of coverage

lines_coverage_size = coverage_information_main.groupby('Test Case')['LINE_COVERED'].sum()
branch_coverage_size = coverage_information_main.groupby('Test Case')['BRANCH_COVERED'].sum()
method_coverage_size = coverage_information_main.groupby('Test Case')['METHOD_COVERED'].sum()

dict_sizes = { 'MethodCoverage': method_coverage_size, 'BranchCoverage': branch_coverage_size , 'LineCoverage': lines_coverage_size }

# Combined DF Coverage Size
df_sizes = pd.DataFrame(dict_sizes)
df_sizes


,MethodCoverage,BranchCoverage,LineCoverage
Test Case,,,
BoundedFIFOTestCase,46,49,221
CategoryTest,63,64,266
CustomLevelTestCase,83,82,339
CyclicBufferTestCase,46,56,224
DOMTestCase,113,126,462
DRFATestCase,125,144,567
ERFATestCase,142,158,634
FileAppenderTest,46,34,204
HTMLLayoutTest,94,119,478


In [21]:
# Calculation of Y-Value

y_value_column_names = df_y_values.columns
df_y_values = df_y_values.fillna("-")

sample_dict = {}


names_of_test_case_classes.sort

for tcs in names_of_test_case_classes:
    for i in range(len(y_value_column_names)):
        tcs_tmp = df_y_values[y_value_column_names[i]].loc[df_y_values[y_value_column_names[i]].values != '-']
        if tcs in df_y_values[y_value_column_names[i]].values:
            if tcs not in sample_dict:
                sample_dict[tcs] = list()
            no_of_tcs_finding_defect = len(tcs_tmp)
            value_of_y_tc = 1/(np.log10(no_of_tcs_finding_defect)+1)
            sample_dict[tcs].append(value_of_y_tc)


pp = pd.DataFrame.from_dict(sample_dict, orient='index')

pp["Y-value"] = pp.sum(axis=1)
pp = pp.reset_index()

pp=pp.rename(columns = {'index':'Test Case'})
df_y_values

,"Fault 1,Fault 2,Fault 3,Fault 4,Fault 5,Fault 6"
0,",,DRFATestCase,,DOMTestCase,DRFATestCase"


In [23]:
# Calculation of TF-IDF Scores for Each Code Class


no_of_classes_impacted_by_each_test_case['totalTestsDivideCoveredTest'] = names_of_test_case_classes.size/(no_of_classes_impacted_by_each_test_case['Test Case'])

no_of_classes_impacted_by_each_test_case['TF-value'] = 1

no_of_classes_impacted_by_each_test_case['IDF'] = np.log10(no_of_classes_impacted_by_each_test_case['totalTestsDivideCoveredTest']) +1 

training_error_best_value = 0
best_threshold = 0

temp_df_idf_for_threshold = no_of_classes_impacted_by_each_test_case.copy()
coverage_information_main.reset_index()
no_of_classes_impacted_by_each_test_case
# Calculation of Similarity Scores - based on the threshold values 
ia = 0
for threshold_val in idf_values:
    df_threshold_idf_condition = pd.DataFrame()
    tf_id_value_df = pd.DataFrame()
    mapping_between_tc_and_code_class = pd.DataFrame()
    merged_inner = pd.DataFrame()
    similarity_score_df = pd.DataFrame()
    df_training_set = pd.DataFrame()
    similarity_score_df = pd.DataFrame()
    df_threshold_idf_condition = temp_df_idf_for_threshold.loc[temp_df_idf_for_threshold['Test Case'] < threshold_val]
    
    # Calculation of Similarity Score 
    
    # A data frame containing IDF and TF Values 
    tf_id_value_df = df_threshold_idf_condition[['CLASS', 'IDF', 'TF-value']]

    # Map each test case vs the code class it covers
    mapping_between_tc_and_code_class = coverage_information_main[['CLASS', 'Test Case']]

    # Merge (inner merge) two data frames (TF/IDF of classes dataframe and mapping between classes and test cases)
    merged_inner = pd.merge(left=mapping_between_tc_and_code_class, right=tf_id_value_df, left_on='CLASS', right_on='CLASS')

    # Data Frame Calculaing Similarity Score of Each Test Case
    similarity_score_df.empty
    similarity_score_df = merged_inner.groupby('Test Case')['IDF'].sum().to_frame().reset_index()
    similarity_score_df.columns = ['Test Case', 'Similarity Score']
    
    df_training_set = pd.DataFrame()
    df_training_set = pd.merge(left=df_sizes, right=similarity_score_df, on='Test Case', how='outer')
    df_training_set= df_training_set.fillna(0)
    #print(" IDF Value = " + str(threshold_val))
    #print(df_training_set)
    
    
    # Add Y-values to the features
    final_set = pd.merge(left=df_training_set, right=pp[['Test Case','Y-value']], on='Test Case', how='outer')

    final_set = final_set.fillna(0)
    #print(final_set)
    linear_regress = LinearRegression()
    # Fit the multiple Linear regression model
    # x is ['Similarity Ratio', 'Coverage Size']

    #print(type(final_set[['MethodCoverage', 'Similarity Score']]))
    #print(type(final_set['Y-value']))
    linear_regress.fit(final_set[['MethodCoverage', 'Similarity Score']], final_set['Y-value'])

    #print("\n\nFOR IDF Threshold = "  + str(threshold_val) + "  Below are Theta Values ")
    
    #Calculating Theta 0, Theta 1, Theta 2.
    
    theta_zero =  linear_regress.intercept_
    theta_one = linear_regress.coef_[0]
    theta_two = linear_regress.coef_[1]
    

    print("Theta 0 = " + str(theta_zero) + "\nTheta 1 = " + str(theta_one) + "\nTheta 2 = " + str(theta_two) )
    
    y_predicted = theta_zero + (theta_one * final_set.MethodCoverage) + (theta_two * final_set['Similarity Score'])
    
    #print(y_predicted)
    training_error = mean_squared_error(final_set['Y-value'], y_predicted)
    #print(training_error)
    print(training_error)
    if training_error_best_value > training_error:
        training_error_best_value = training_error
        print(training_error)
        print(training_error_best_value)
        
        best_threshold = threshold_val
    elif training_error_best_value == 0:
        training_error_best_value = training_error

print("Least Training Error (lowest value) is = " + str(training_error_best_value))
print(" For Threshold = " + str(best_threshold))

Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Theta 0 = 0.0
Theta 1 = 0.0
Theta 2 = 0.0
0.0
Least Training Error (lowest value) is = 0.0
 For Threshold = 0


In [15]:
linear_regress.score(final_set[['MethodCoverage', 'Similarity Score']], final_set['Y-value'])

1.0

In [9]:
# BACKUP IMPLEMENTAION - DO NOT TOUCH
# Calculation of Similarity Score Backup 
# A data frame containing IDF and TF Values 
#tf_id_value_df = no_of_classes_impacted_by_each_test_case[['CLASS', 'IDF', 'TF-value','Test Case']]

#tf_id_value_df
# Calculation of Y-values


# Map each test case vs the code class it covers
#coverage_information_main.reset_index()
#mapping_between_tc_and_code_class = coverage_information_main[['CLASS', 'Test Case']]

#mapping_between_tc_and_code_class.loc[mapping_between_tc_and_code_class['Test Case'] == 'ApplicationTest']


# Merge (inner merge) two data frames (TF/IDF of classes dataframe and mapping between classes and test cases)
#merged_inner = pd.merge(left=mapping_between_tc_and_code_class, right=tf_id_value_df, left_on='CLASS', right_on='CLASS')
#aa = merged_inner.loc[merged_inner['Test Case_y'] < 14]
#aa.loc[aa['Test Case_x'] == 'ApplicationTest']
# Data Frame Calculaing Similarity Score of Each Test Case
#similarity_score_df = merged_inner.groupby('Test Case')['IDF'].sum().to_frame().reset_index()
#similarity_score_df.columns = ['Test case', 'Similarity Score']
#similarity_score_df


#OBSOLETE: Calculation of Y-Value

#df_y_values = pd.read_csv('y-values.csv')
#y_value_column_names = df_y_values.columns
#df_y_values = df_y_values.fillna("-")

#sample_dict = {}
#names_of_test_case_classes.sort

#for tcs in names_of_test_case_classes:
    #for i in range(len(y_value_column_names)):
       # tcs_tmp = df_y_values[y_value_column_names[i]].loc[df_y_values[y_value_column_names[i]].values != '-']
      #  if tcs in df_y_values[y_value_column_names[i]].values:
     #       if tcs not in sample_dict:
    #            sample_dict[tcs] = list()
   #         no_of_tcs_finding_defect = len(tcs_tmp)
  #          value_of_y_tc = 1/(np.log10(no_of_tcs_finding_defect)+1)
 #           sample_dict[tcs].append(value_of_y_tc)
#

#pp = pd.DataFrame.from_dict(sample_dict, orient='index')

#pp["Y-value"] = pp.sum(axis=1)

# Linear Regression

#linear_regress = LinearRegression()
# Fit the multiple Linear regression model
# x is ['Similarity Ratio', 'Coverage Size']
#linear_regress.fit(x,y)

# Three co-efficients 
#linear_regress.coef_

In [10]:
final_set.MethodCoverage


final_set['Similarity Score']

final_set


,Test Case,MethodCoverage,BranchCoverage,LineCoverage,Similarity Score,Y-value
0,ApplicationTest,97,172,395,6.300783,3.082338
1,ArgumentIsOptionTest,54,84,184,1.215600,0.959540
2,BasicParserTest,62,92,199,7.316578,1.583736
3,BugCLI133Test,28,18,82,1.215600,0.000000
4,BugCLI13Test,69,69,220,1.215600,0.459540
5,BugCLI148Test,63,70,216,1.215600,0.459540
6,BugCLI162Test,42,72,182,5.085184,1.265584
7,BugCLI18Test,30,74,166,3.422426,0.588592
8,BugCLI71Test,56,74,179,4.285843,0.459540
9,BugsTest,120,201,481,11.033784,2.760920
